Plan for steps in this processing code
1. Define a base grid
2. Load all the tif files - know how
4. Load the shp files - know how
5. Project the shp files to tif - at the resolution / bounds, etc with the base grid - know how
6. Generate the train deposit / occurence tif files - know how
6. Unify all the tif data - know how

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import rasterio

import utilities as utils

In [ ]:
tifs, shps = utils.get_input_var_files("Australia")

Loads the raster data

In [ ]:
rasters = utils.load_rasters(tifs, verbosity=1)

Loads rasters of the vector data if available; otherwise generates them

In [ ]:
try:
    rasters += utils.load_rasters(shps, verbosity=1)
except rasterio.RasterioIOError:
    base_raster = rasters[-1] # defaults to intermediate resolution raster
    vectors = utils.load_vectors(shps, verbosity=0)
    pbar = tqdm(zip(shps, vectors))
    for shp, vector in pbar:
        pbar.set_description(f"Processing {shp}")
        utils.proximity_raster_of_vector_points(base_raster, shp, vector)
    rasters += utils.load_rasters(shps, verbosity=1)

In [ ]:
bottom = left = 180.0
top = right = -180.0
for raster in rasters:
    if raster.bounds.left < left:
        left = raster.bounds.left
    if raster.bounds.bottom < bottom:
        bottom = raster.bounds.bottom
    if raster.bounds.top > top:
        top = raster.bounds.top
    if raster.bounds.right > right:
        right = raster.bounds.right
print(f"Computed bounds - left {left},right {right},top {top},bottom {bottom}\n")

In [ ]:
region_rect = S2LatLngRect(
    S2LatLng.FromDegrees(-51.264871, -30.241701),
    S2LatLng.FromDegrees(-51.04618, -30.000003))
coverer = S2RegionCoverer()
coverer.set_min_level(8)
coverer.set_max_level(15)
coverer.set_max_cells(500)
covering = coverer.GetCovering(region_rect)


In [ ]:
x, y, interpolated_values = simple_idw(deep_mag_worms_aus, "Steepness", (0.2, 0.2))

In [ ]:
plt.imshow(interpolated_values, cmap='jet', extent=[x.min(), x.max(), y.min(), y.max()])
plt.colorbar()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('IDW Interpolation')
plt.show()

In [ ]:
arr = aus_rasters[1].read(1, masked=False)
plt.imshow(arr, cmap='gray')
plt.colorbar()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('AUS Moho - without mask')
plt.show()

arr = aus_rasters[1].read(1, masked=True)
plt.imshow(arr, cmap='gray')
plt.colorbar()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('AUS Moho - with mask')
plt.show()

In [ ]:
# -*- coding: utf-8 -*-

# Simple s2 KML generator
# Copyright 2018, Sean C. Nichols
#  seanni@trichotomy.ca

# Hereby released into the public domain: feel free to abuse this code as you see fit!
#

# usage: python s2gen.py <max_lat> <min_lat> <max_lng> <min_lng> <s2_lvl>
#
# (i.e.: N-S-E-W order)
#

from __future__ import print_function

from sys import argv, stderr

from s2sphere import RegionCoverer, Cell, LatLng, LatLngRect

def usage(msg=None):
    if msg:
        stderr.write(msg + '\n\n')
    stderr.write('usage: {} <max_lat> <min_lat> <max_lng> <min_lng> <s2_lvl>\n'.format(argv[0]))
    exit(1)

def swap_latlong(latlong):
    return '{},{}'.format(*latlong.split(',')[::-1])

def s2gen(max_lat, min_lat, max_lng, min_lng, s2_lvl):
    point_nw = LatLng.from_degrees(max_lat, min_lng)
    point_se = LatLng.from_degrees(min_lat, max_lng)

    rc = RegionCoverer()
    rc.min_level = s2_lvl
    rc.max_level = s2_lvl
    rc.max_cells = 1000000

    cellids = rc.get_covering(LatLngRect.from_point_pair(point_nw, point_se))

    kml = (
        '<?xml version="1.0" encoding="UTF-8"?>\n'
        '<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:kml="http://www.opengis.net/kml/2.2" xmlns:atom="http://www.w3.org/2005/Atom">\n'
        '<Document><name>{0}</name>\n'
        '<Style id="s2_poly_style"><LineStyle><color>ff0000ff</color><width>2</width></LineStyle><PolyStyle><fill>0</fill></PolyStyle></Style>\n'
        '<Folder><name>{0}</name><open>1</open>'
    ).format('S2 cells level ' + str(s2_lvl))
    for cid in cellids:
        vertices = [LatLng.from_point(Cell(cid).get_vertex(v)) for v in range(4)]
        kml_coords = ['{},0'.format(swap_latlong(str(v).split()[-1])) for v in vertices]
        kml += (
            '<Placemark><name>{}</name><styleUrl>#s2_poly_style</styleUrl><Polygon><tessellate>1</tessellate><outerBoundaryIs><LinearRing>'
            '<coordinates>{} {}</coordinates>'
            '</LinearRing></outerBoundaryIs></Polygon></Placemark>'
        ).format(cid.id(), ' '.join(kml_coords), kml_coords[0])
    kml += (
        '</Folder></Document>\n'
        '</kml>'
    )

    return kml

kml_doc = s2gen(-43.65, -10.049999999999997, 153.65, 112.95, 12)
with open("s2_cell.kml", "w") as f:
    f.write(kml_doc)